In [1]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
from astropy import units as u
from gwpy.detector import Channel
from gwpy.timeseries import TimeSeries, TimeSeriesList, TimeSeriesDict
from gwexpy.types import MetaData, MetaDataDict, MetaDataMatrix, SeriesMatrix

# MetaData

In [2]:
entry = MetaData(
    name="IFO_X",
    channel="K1:PEM-EX_ACC",
    unit=u.Hz
)

print(entry)             # dict 表示
print(entry.name)        # 属性アクセス
print(entry["unit"])     # dictアクセス
print(entry.channel.name)  # Channelオブジェクトのプロパティ


    name: IFO_X	    unit: Hz	 channel: K1:PEM-EX_ACC
IFO_X
Hz
K1:PEM-EX_ACC


In [3]:
m1 = MetaData(name="a", unit="m")
m2 = MetaData(name="b", unit="m")
m3 = MetaData(name="c", unit="s")

print("m1, m2, m3 : MetaData object")
print("abs(m1) →", np.abs(m1))
print("m1 * u.kg →", m1*u.kg)
print("m1 * (5*u.kg) →", m1*(5*u.kg))
print("m1 + m2 →", np.add(m1, m2))
print("m1 * m3 →", np.multiply(m1, m3))
print("m1 / m3 →", np.divide(m1, m3))
print("m1 ** 2 →", np.power(m1, 2))
print("np.sqrt(m1) →", np.sqrt(m1))

try:
    np.add(m1, m3)
except Exception as e:
    print("Expected unit mismatch:", e)

try:
    np.log(m1)
except Exception as e:
    print("Expected dimensionless error:", e)

m1, m2, m3 : MetaData object
abs(m1) →     name: a	    unit: m	 channel: 
m1 * u.kg →     name: a	    unit: kg m	 channel: 
m1 * (5*u.kg) →     name: a	    unit: kg m	 channel: 
m1 + m2 →     name: a	    unit: m	 channel: 
m1 * m3 →     name: a	    unit: m s	 channel: 
m1 / m3 →     name: a	    unit: m / s	 channel: 
m1 ** 2 →     name: a	    unit: m2	 channel: 
np.sqrt(m1) →     name: a	    unit: m(1/2)	 channel: 
Expected unit mismatch: add requires compatible units
Expected dimensionless error: log requires dimensionless input


In [4]:
m1 == m1

True

# MetaDataDict

In [5]:
rows = MetaDataDict([
    {"name": "IFO_X", "channel": "K1:PEM-EX_ACC", "unit": u.Hz},
    {"name": "IFO_Y", "channel": "K1:PEM-EY_ACC", "unit": u.Hz}
])

print(rows)                  # dict表示
print(rows['key0'].channel)       # Channel object
print(rows.names)            # ['IFO_X', 'IFO_Y']
print(rows.units)            # [Unit("Hz"), Unit("Hz")]


       name        channel unit
key                            
key0  IFO_X  K1:PEM-EX_ACC   Hz
key1  IFO_Y  K1:PEM-EY_ACC   Hz
K1:PEM-EX_ACC
['IFO_X', 'IFO_Y']
[Unit("Hz"), Unit("Hz")]


In [6]:
df = rows.to_dataframe()
display(df)  # Jupyterなら表として表示される

# 保存して確認（ファイル書き込み不要なら省略可）
rows.write("rows.csv")

# 読み込みから再構成
rows2 = MetaDataDict.read("rows.csv")
display(rows2.to_dataframe())


,name,channel,unit
key,,,
key0,IFO_X,K1:PEM-EX_ACC,Hz
key1,IFO_Y,K1:PEM-EY_ACC,Hz


,name,channel,unit
key,,,
key0,IFO_X,K1:PEM-EX_ACC,Hz
key1,IFO_Y,K1:PEM-EY_ACC,Hz


In [7]:
df = pd.DataFrame([
    {"name": "IFO_X", "channel": "K1:PEM-EX_ACC", "unit": "Hz"},
    {"name": "IFO_Y", "channel": "K1:PEM-EY_ACC", "unit": "Hz"},
])
df.index = ["x", "y"]  # キーを文字列で与える

rows3 = MetaDataDict(df)
display(rows3)

,name,channel,unit
key,,,
x,IFO_X,K1:PEM-EX_ACC,Hz
y,IFO_Y,K1:PEM-EY_ACC,Hz


In [8]:
# TimeSeries を作成
ts = TimeSeries([0.1, 0.2, 0.3], dt=1/1000, unit=u.Hz, name="IFO_X")
ts.channel = "K1:PEM-EX_ACC"  # GWpy の仕様では手動で付与する必要あり

# メタデータ化
meta = MetaData.from_series(ts)

# 確認
print("name:", meta.name)
print("unit:", meta.unit)
print("channel:", meta.channel)


name: IFO_X
unit: Hz
channel: K1:PEM-EX_ACC


In [9]:
# 複数 TimeSeries
ts_list = {
    "IFO_X": TimeSeries([1, 2, 3], dt=1/1024, unit="Hz", name="IFO_X", channel= "K1:PEM-EX_ACC"),
    "IFO_Y": TimeSeries([4, 5, 6], dt=1/1024, unit="Hz", name="IFO_Y", channel= "K1:PEM-EY_ACC"),
}


# メタデータ辞書に変換
meta_dict = MetaDataDict.from_series(ts_list)

# 表示
print(meta_dict)
meta_dict.to_dataframe()


標準 dict は順序が保証されないため注意して使用してください
        name        channel unit
key                             
IFO_X  IFO_X  K1:PEM-EX_ACC   Hz
IFO_Y  IFO_Y  K1:PEM-EY_ACC   Hz


,name,channel,unit
key,,,
IFO_X,IFO_X,K1:PEM-EX_ACC,Hz
IFO_Y,IFO_Y,K1:PEM-EY_ACC,Hz


In [10]:
# 単体
meta = MetaData.from_series(ts)
print(meta)

# 複数（dict）
meta_dict = MetaDataDict.from_series({
    "IFO_X": TimeSeries([1, 2, 3], dt=1/1024, unit="Hz", name="IFO_X", channel= "K1:PEM-EX_ACC"),
    "IFO_Y": TimeSeries([4, 5, 6], dt=1/1024, unit="Hz", name="IFO_Y", channel= "K1:PEM-EY_ACC"),
})
print(meta_dict)

# 複数（list）
meta_dict = MetaDataDict.from_series([TimeSeries([1, 2, 3], dt=1/1024, unit="Hz", name="IFO_X", channel= "K1:PEM-EX_ACC"),
                                      TimeSeries([4, 5, 6], dt=1/1024, unit="Hz", name="IFO_Y", channel= "K1:PEM-EY_ACC")])
print(meta_dict)

# 複数（dict）
meta_dict = MetaDataDict.from_series(TimeSeriesDict({
    "IFO_X": TimeSeries([1, 2, 3], dt=1/1024, unit="Hz", name="IFO_X", channel= "K1:PEM-EX_ACC"),
    "IFO_Y": TimeSeries([4, 5, 6], dt=1/1024, unit="Hz", name="IFO_Y", channel= "K1:PEM-EY_ACC"),
}))
meta_dict

    name: IFO_X	    unit: Hz	 channel: K1:PEM-EX_ACC
標準 dict は順序が保証されないため注意して使用してください
        name        channel unit
key                             
IFO_X  IFO_X  K1:PEM-EX_ACC   Hz
IFO_Y  IFO_Y  K1:PEM-EY_ACC   Hz
標準 dict は順序が保証されないため注意して使用してください
       name        channel unit
key                            
key0  IFO_X  K1:PEM-EX_ACC   Hz
key1  IFO_Y  K1:PEM-EY_ACC   Hz
標準 dict は順序が保証されないため注意して使用してください


,name,channel,unit
key,,,
IFO_X,IFO_X,K1:PEM-EX_ACC,Hz
IFO_Y,IFO_Y,K1:PEM-EY_ACC,Hz


In [11]:
md1 = MetaDataDict([{"name": "a", "unit": "m"}, {"name": "b", "unit": "m"}])
md2 = MetaDataDict([{"name": "a", "unit": "m"}, {"name": "b", "unit": "m"}])

print("md1, md2 : MetaDataDict object")

print("\n md1 + md2\n",md1 + md2)
print("\n md1 - md2\n",md1 - md2)
print("\n md1 * md2\n",md1 * md2)
print("\n md1 / md2\n",md1 / md2)
print("\n abs(md1)\n",abs(md1))
print("\n md1**4\n",md1**4)
print("\n np.sqrt(md1)\n",np.sqrt(md1))

print("\n md1*u.kg\n",md1*u.kg)
print("\n md1*(5*u.kg)\n",md1*(5*u.kg))

md1, md2 : MetaDataDict object

 md1 + md2
      name channel unit
key                   
key0    a            m
key1    b            m

 md1 - md2
      name channel unit
key                   
key0    a            m
key1    b            m

 md1 * md2
      name channel unit
key                   
key0    a           m2
key1    b           m2

 md1 / md2
      name channel unit
key                   
key0    a             
key1    b             
標準 dict は順序が保証されないため注意して使用してください

 abs(md1)
      name channel unit
key                   
key0    a            m
key1    b            m

 md1**4
      name channel unit
key                   
key0    a           m4
key1    b           m4
標準 dict は順序が保証されないため注意して使用してください

 np.sqrt(md1)
      name channel    unit
key                      
key0    a          m(1/2)
key1    b          m(1/2)

 md1*u.kg
      name channel  unit
key                    
key0    a          kg m
key1    b          kg m

 md1*(5*u.kg)
      name channel  unit
key      

# MetaDataMatrix

In [12]:
# 初期メタ情報（2x2）
data = [
    [{"name": "a", "unit": "Hz"}, {"name": "b", "unit": "Hz"}],
    [{"name": "c", "unit": "s"},  {"name": "d", "unit": "s"}]
]

mdmat = MetaDataMatrix(data)

# 確認
print("Shape:", mdmat.shape)
print("mdmat[0, 0]:", mdmat[0, 0])
print("Unit of [1, 1]:", mdmat[1, 1].unit)
mdmat

Shape: (2, 2)
mdmat[0, 0]:     name: a	    unit: Hz	 channel: 
Unit of [1, 1]: s


,name,unit,channel,row,col
0,a,Hz,,0,0
1,b,Hz,,0,1
2,c,s,,1,0
3,d,s,,1,1


In [13]:
print("Name matrix:")
print(mdmat.names)

print("Unit matrix:")
print(mdmat.units)


Name matrix:
[['a' 'b']
 ['c' 'd']]
Unit matrix:
[[Unit("Hz") Unit("Hz")]
 [Unit("s") Unit("s")]]


In [14]:
# すべてのセルに同じMetaDataを埋める
mdmat.fill(MetaData(name="x", unit=u.Hz))

print("Filled name matrix:")
print(mdmat)


Filled name matrix:
  name unit channel  row  col
0    x   Hz            0    0
1    x   Hz            0    1
2    x   Hz            1    0
3    x   Hz            1    1


In [15]:
# DataFrameに変換して表示
df = mdmat.to_dataframe()
print("As DataFrame:")
print(df)

# CSVに書き出し
mdmat.write("meta_test.csv")

# 読み込んで復元
mdmat2 = MetaDataMatrix.read("meta_test.csv")

# 確認
print("Restored matrix:")
mdmat2

As DataFrame:
  name unit channel  row  col
0    x   Hz            0    0
1    x   Hz            0    1
2    x   Hz            1    0
3    x   Hz            1    1
Restored matrix:


,name,unit,channel,row,col
0,x,Hz,nan,0,0
1,x,Hz,nan,0,1
2,x,Hz,nan,1,0
3,x,Hz,nan,1,1


In [16]:
print("=== MetaDataMatrix math test ===")

# --- MetaDataMatrix を生成 ---
m1 = MetaData(name="a", unit="m")
m2 = MetaData(name="b", unit="m")
m3 = MetaData(name="c", unit="s")

mdm1 = MetaDataMatrix([[m1, m2], [m1, m2]])
mdm2 = MetaDataMatrix([[m2, m1], [m2, m1]])
mdm3 = MetaDataMatrix([[m3, m3], [m3, m3]])


print("mdm1, mdm2, mdm3 : MetaDataMatrix object")

# --- 加算（同じ単位） ---
print("\nmdm1 + mdm2")
print(mdm1 + mdm2)

# --- 乗算（単位同士の積） ---
print("\nmdm1 * mdm3")
print(mdm1 * mdm3)

# --- 除算（単位同士の商） ---
print("\nmdm1 / mdm3")
print(mdm1 / mdm3)

# --- スカラーとの乗算 ---
print("\nmdm1 * 3")
print(mdm1 * 3)

# --- スカラーべき乗 ---
print("\nmdm1 ** 2")
print(mdm1 ** 2)

# --- 単位との乗算 ---
print("\nmdm1 * u.kg")
print(mdm1 * u.kg)

# --- スカラー物理量との乗算 ---
print("\nmdm1 * (5*u.kg)")
print(mdm1 * (5*u.kg))


# --- 絶対値 ---
print("\nabs(mdm1)")
print(abs(mdm1))

# --- 平方根 ---
print("\nnp.sqrt(mdm1)")
print(np.sqrt(mdm1))

# --- 単位不一致の加算（例外チェック） ---
print("\nExpected error:")
try:
    print(mdm1 + mdm3)
except Exception as e:
    print("→", type(e).__name__, ":", e)

=== MetaDataMatrix math test ===
mdm1, mdm2, mdm3 : MetaDataMatrix object

mdm1 + mdm2
  name channel unit  row  col
0    a            m    0    0
1    b            m    0    1
2    a            m    1    0
3    b            m    1    1

mdm1 * mdm3
  name channel unit  row  col
0    a          m s    0    0
1    b          m s    0    1
2    a          m s    1    0
3    b          m s    1    1

mdm1 / mdm3
  name channel   unit  row  col
0    a          m / s    0    0
1    b          m / s    0    1
2    a          m / s    1    0
3    b          m / s    1    1

mdm1 * 3
  name channel unit  row  col
0    a            m    0    0
1    b            m    0    1
2    a            m    1    0
3    b            m    1    1

mdm1 ** 2
  name channel unit  row  col
0    a           m2    0    0
1    b           m2    0    1
2    a           m2    1    0
3    b           m2    1    1

mdm1 * u.kg
  name channel  unit  row  col
0    a          kg m    0    0
1    b          kg m    0    1


In [17]:

# 2×3のメタデータ配列を生成（内容は適当でOK）
meta_array = [[MetaData(unit="m", name=f"s{i}{j}", channel=f"ch{i}{j}") for j in range(3)] for i in range(2)]

# 行ラベル・列ラベル未指定（自動生成されるはず）
meta = MetaDataMatrix(meta_array)

print("行ラベル（自動生成）:", meta.row_keys)
print("列ラベル（自動生成）:", meta.col_keys)

# --- 期待される出力 ---
# 行ラベル（自動生成）: ['row0', 'row1']
# 列ラベル（自動生成）: ['col0', 'col1', 'col2']

行ラベル（自動生成）: ['row0', 'row1']
列ラベル（自動生成）: ['col0', 'col1', 'col2']
